In [1]:
import folium
import pandas as pd
# import geopandas as gpd 
from folium.features import DivIcon
from folium import plugins
# from scipy.ndimage import imread
from osgeo import gdal
import cv2
from folium.raster_layers import ImageOverlay
import rasterio as rio

In [2]:
df = pd.read_csv('ship_det_LL.csv')
ais = pd.read_csv('ShipTest/esri/18072020.csv')
ais = ais.drop_duplicates(subset='mmsi')

In [3]:
# m=folium.Map(location=[43.7, 9.], zoom_start=9.,tiles='Stamen Terrain')
# #                        Y   , X
# # Add detections
# for i in range(len(df)):

#     x, y = float(df.iloc[i].latitude), float(df.iloc[i].longitude)
#     if (abs(x-43.7) < 0.55) & (abs(y-9.)<0.55):
#             folium.Marker(
#                 location=[x,y],
#                 popup=f"Ship Detected {i}",
#                 icon=folium.Icon(icon='fa-ship', prefix='fa'),
#                 ).add_to(m)
# m


In [4]:
driver=gdal.GetDriverByName('GTiff')
driver.Register() 
#Open raster file
ds = gdal.Open('ShipTest/S1A_IW_GRDH_1SDV_20200718T171436_20200718T171501_033512_03E220_DEFC_TC.tif') 
if ds is None:
    print('Could not open')

#Get coordinates, cols and rows
geotransform = ds.GetGeoTransform()
cols = ds.RasterXSize 
rows = ds.RasterYSize 

#Get extent
xmin=geotransform[0]
ymax=geotransform[3]
xmax=xmin+cols*geotransform[1]
ymin=ymax+rows*geotransform[5]

#Get Central point
centerx=(xmin+xmax)/2
centery=(ymin+ymax)/2

#Raster convert to array in numpy
bands = ds.RasterCount
band=ds.GetRasterBand(1)
dataset= band.ReadAsArray(0,0,cols,rows)

im=(dataset.astype('uint8')+20).astype('uint8')
r,c = im.shape
saved_im = cv2.resize(im, [r//3,c//3])

In [5]:
#Visualization in folium
m=folium.Map(location=[centery, centerx], zoom_start=9.2,tiles='Stamen Terrain')

# Add Raster
ImageOverlay(
    image=saved_im,
    bounds=[[ymin, xmin], [ymax, xmax]],
).add_to(m)

# Add detections
for i in range(len(df)):
    x, y = float(df.iloc[i].latitude), float(df.iloc[i].longitude)
    if (abs(x-43.7) < 0.55) & (abs(y-9.)<0.55):
            folium.Marker(
                location=[x,y],
                popup=f"Ship Detected {i}",
                icon=folium.Icon(icon='fa-ship', prefix='fa'),
                ).add_to(m)
# Add AIS
for i in range(len(ais)):
    text='mmsi: '+str(ais.iloc[i].mmsi)
    folium.CircleMarker(location=[float(ais.iloc[i].latitude.replace(',','.')),float(ais.iloc[i].longitude.replace(',','.'))], 
                        radius=10,
                        line_color='white', 
                        fill_color='white', 
                        popup='mmsi: '+str(ais.iloc[i].mmsi),
                        fill_opacity=1,
                        show=True).add_to(m)
    folium.map.Marker(
        [float(ais.iloc[i].latitude.replace(',','.')),float(ais.iloc[i].longitude.replace(',','.'))],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 12pt; color: white">%s</div>' % text,
            )
        ).add_to(m)



m.save("Mappa.html")
print('Done')

Done
